In [1]:
#importing the required modules and classes
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math

In [2]:
iris = datasets.load_iris() # loading iris dataframe

#  converting dataset to pandas DataFrame
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
# naming the column headers of dataframe
df.columns = ['sl' , 'sw' , 'pl' , 'pw' , 'target']

#  splitting dataset into train and test data for prediction
x_train , x_test = train_test_split(df.iloc[: , 0:-1] , random_state = 1)
y_train , y_test = train_test_split(df.iloc[: , -1] , random_state = 1)
list_of_features = x_train.columns.tolist() #making a list of features using toList method

index = [i for i in range(len(x_train.columns))]


In [3]:
#defining entropy function for decision  tree
def CalculateEntropy(y):
    values = set(y) # storing all possible values of target in a set 
    total = len(y) # storing total no. of datasets (rows) for calculating probability
    entropy1 = 0   #initially entropy of one level is zero
    for l in values: # iterating on each element of set 
        p = (y == l).sum() / total # calculating probability then finding all no. of rows with
                                                   # and dividing it with total that gives probability
        entropy1 += -p*math.log2(p)  
    # by using formula (-1) * probability * log base 2(probability)
    return entropy1

In [4]:
# calculating information gain for decision tree
def info_gain(x , y , index): # received dataframe and index of feature whoose information gain is to be returned
    
    
    initial_entropy = CalculateEntropy(y) #initialising entropy
    step = x[x.columns[index]].mean() #step is equal to the mean of all the values of the column
    init = step/2  # initial is represented by init 
    condition = 0 # condition is initially taken to be zero
    maxm = -100000000 #a random high negative value is chosen
    split_info_max = -100000000000000 #same
    while init <= 2*step : # while loop runs 
        final_entropy = 0 #final entropy is updated in the loop
        y_new_greater  = y.loc[x[x.columns[index]] >= init] #further below are
        y_new_less  = y.loc[x[x.columns[index]] < init]     #calculations for selecting the best feature
        final_entropy = (CalculateEntropy(y_new_greater)) + (CalculateEntropy(y_new_less))
        if final_entropy > maxm:
            maxm = final_entropy #updating maximum entropy 
            condition = init
            split_info_max = split_info(y_new_greater , y_new_less) 
        init += init #updating init
    return maxm - CalculateEntropy(y), condition , split_info_max

In [5]:
def split_info(y1 , y2) :
    d1 = len(y1) #storing length of y_new_greater
    d2 = len(y2) #storing length of y_new_less
    d = d1 + d2
    if(d1 == 0 or d2 == 0 or d == 0): #adding a condition to check null value
        return 0
    # adding final split value for each possible value of feature by (-1) * (d1/d) * log base 2 (d1/d)f
    p = (-1*d1*math.log2(d1/d))/d +  (-1*d2*math.log2(d2/d))/d
    return p

In [6]:
def gain_ratio(x ,y ,index): # received dataframe and index of feature whoose information gain is to be returned
    return info_gain(x , y , index)[0] / info_gain(x , y , index)[2] #returning the information gain ratio

In [7]:
def decisionTree(x , y , feature_index,level): # received x_train, y_train 
    # and index of feature 
    
    
## case one : when leaf node is encountered
    if(len(set(y)) == 1): 
        print('Level : ' , level)
        print('Current entropy : ' , 0.0)
        print('count of feature :' , len(y))
        print('Leaf node has been reached')
        print()
        return
    
## case two : when all the features have been used up

    if(len(feature_index) == 0):
        print('Level : ' , level)
        print('Current entropy : ' , CalculateEntropy(y))
        print('leaf node reached')
        print()
        return
    
## normal case

    maxm = -10000000
    best_feature = -1
    condition = 0
    for i in feature_index:
        gain = gain_ratio(x , y , i)
        if gain > maxm:
            maxm = gain
            best_feature = i
            condition = info_gain(x , y , i)[1]
    
    
    print('Level : ' , level)
    print('Current entropy : ' , CalculateEntropy(y))
    print('Splitting on feature' , x.columns[best_feature], 'with gain ratio : ' , maxm)
    print('number of features :' , len(y))
    print()
    
    ## selecting the best feature process
    labels = set(x[x.columns[best_feature]])
    
    if best_feature not in feature_index: # check if feature is used
        return
    feature_index.remove(best_feature)    # remove if yes
    
    x_new_greater  = x.loc[x[x.columns[best_feature]] >= condition]
    y_new_greater  = y.loc[x[x.columns[best_feature]] >=  condition]
    x_new_lesser  = x.loc[x[x.columns[best_feature]] < condition]
    y_new_lesser  = y.loc[x[x.columns[best_feature]] <  condition]
    
    #recursively calling decision tree fnction 
    decisionTree(x_new_greater , y_new_greater , feature_index , level + 1)
    decisionTree(x_new_lesser , y_new_lesser , feature_index , level + 1)
    feature_index.append(best_feature)

In [8]:
decisionTree(x_train , y_train , index,0)

C:\Users\Rajinder\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Level :  0
Current entropy :  1.58071971384221
Splitting on feature sw with gain ratio :  1.179883386144517
number of features : 112

Level :  1
Current entropy :  1.3246095799115576
Splitting on feature pl with gain ratio :  0.1958007564964672
number of features : 48

Level :  2
Current entropy :  1.516468623731106
Splitting on feature pw with gain ratio :  -10000000
number of features : 28

Level :  2
Current entropy :  0.0
count of feature : 20
Reached leaf node

Level :  1
Current entropy :  1.4185644431995963
Splitting on feature sl with gain ratio :  0.9774236456008285
number of features : 64

Level :  2
Current entropy :  0.9366673818775625
Splitting on feature pl with gain ratio :  0.03507528915710396
number of features : 34

Level :  3
Current entropy :  0.0
count of feature : 14
Reached leaf node

Level :  3
Current entropy :  0.9709505944546686
Splitting on feature pw with gain ratio :  0.25705233941428574
number of features : 20

Level :  4
Current entropy :  0.591672778582